# Welcome to Hanasu

In [ ]:
import zipfile
with zipfile.ZipFile('wavs.zip', 'r') as zip_ref: zip_ref.extractall('zipfolder')

In [ ]:
%cd /content/drive/MyDrive/hanasu/hanasu
!pip install -r requirements.txt
%cd /content/drive/MyDrive/vits2_pytorch-main/monotonic_align
!python setup.py build_ext --inplace
%cd ../
!apt-get update && apt-get install -y espeak-ng
!pip install phonemizer

# Preprocess the dataset

In [ ]:
from text import preprocess_filelists
preprocess_filelists(["transcript.txt"])

# Fix Bugs

In [ ]:
!pip install librosa==0.9.1
!pip install tensorboard==2.12.0 tensorflow==2.12.0
!pip install matplotlib==3.7.0 # This fixes: 'FigureCanvasAgg' object has no attribute 'tostring_rgb'
!pip install numpy==1.26.4

# Training

In [ ]:
!bash train.sh /content/Yuna/config.json 1

# Inference

In [ ]:
%cd ../hanasu

In [ ]:
from models import inference, load_model
from scipy.io.wavfile import write
import sounddevice as sd

In [ ]:
text ="Text for generation."
# Or you can read from a file:
# with open("../output.txt", 'r', encoding='utf-8') as f: text = f.read()

In [ ]:
model = load_model("./configs/config.json", "/Users/yuki/Downloads/G_158000.pth", device="mps")

In [ ]:
# No streaming inference
result = inference(
    model=model,
    text=text,
    noise_scale=0.2,
    noise_scale_w=1.0,
    length_scale=1.0,
    device="mps",
    stream=False,
)

write(data=result, rate=48000, filename="sample_vits2.wav")

In [ ]:
# Streaming inference
audio_generator = inference(
    model=model,
    text=text,
    noise_scale=0.17,
    noise_scale_w=1.0,
    length_scale=1.0,
    device="mps",
    stream=True,
)

for audio_chunk in audio_generator:
    # Play each chunk immediately as it's generated
    sd.play(audio_chunk, samplerate=48000)
    sd.wait()  # Wait for chunk to finish playing
    print(f"Played chunk of {len(audio_chunk)} samples")